In [1]:
# Data
import pandas as pd
import numpy as np

import names # My globals file
import random
import pickle

from datetime import datetime, timedelta

# Testing and processing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit

# Scoring
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Models
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# Plotting
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def get_next_weekday(startdate, weekday):
    '''
    @startdate: given date, in format '2013-05-25'
    @weekday: week day as a integer, between 0 (Monday) to 6 (Sunday)
    '''
    
    d = datetime.strptime(startdate, '%Y-%m-%d')
    t = timedelta((7 + weekday - d.weekday()) % 7)
    
    return (d + t).strftime('%Y-%m-%d')


def custom_insert(l, index, element):
    l.insert(index, element)
    return l


def get_ts(df, periods):
    final_df = pd.DataFrame(index = df.index)
    for state in names.state_names:
        state_df = df[df['location_name'] == state]

        for col in list(df)[2:-4]:
            for i in range(1, periods + 1):
                state_df[col + '_t-' + str(i)] = state_df[col].shift(i)

        final_df = final_df.append(state_df)

    final_df = final_df[final_df['location_name'].notna()]
    non_claims = [col for col in list(final_df) if col not in names.claim_cols]
    final_df[non_claims] = final_df[non_claims].fillna(0)
    
    return final_df


def rmse(y_true, y_pred):
    return np.sqrt(((y_true - y_pred) ** 2).mean())


def string_fixer(s):
    if type(s) == str:
        s = s.replace(',', '').replace('(', '').replace(')', '')
        
        if s.strip() in ['Z', 'D']:
            s = '0'
    
    return s

In [3]:
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    df = pd.DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = pd.concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df


# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)


# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]


# scale train and test data to [-1, 1]
def scale(train, test):
    # fit scaler
    scaler = StandardScaler()
    scaler = scaler.fit(train)
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    
    if len(test) > 0:
        # transform test
        test = test.reshape(test.shape[0], test.shape[1])
        test_scaled = scaler.transform(test)
    else:
        test_scaled = []

    return scaler, train_scaled, test_scaled


# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]


# fit an LSTM network to training data
def fit_lstm(model, train, batch_size, nb_epoch):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])

    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()

    return model


def get_lstm(batch_size, neurons, shape):
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, 1, shape), stateful=True))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model


# make a one-step forecast
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]


def train_model(training_features, initial_claims, testing_n):
    test_sets = {}
    validation = {}
    results = {}
    scalers = {}

    shape = training_features.shape[1]
    
    lstm_model = get_lstm(1, 4, shape)

    states = training_features['location_name'].unique()
    random.shuffle(states)

    for i, state in enumerate(states):
        features = training_features[training_features['location_name'] == state].drop('location_name', axis = 1).reset_index(drop = True)
        series = initial_claims[training_features['location_name'] == state]

        # transform data to be stationary
        raw_values = series.values
        diff_values = difference(raw_values, 1)
        
        # transform data to be supervised learning
        supervised = timeseries_to_supervised(diff_values, 1)
        supervised_values = features.drop(
            len(features) - 1).reset_index(
            drop = True).join(supervised, 
                              how = 'left').values

        # split data into train and test-sets
        if testing_n != 0:
            train, test = supervised_values[0:-testing_n], supervised_values[-testing_n:]
        else:
            train = supervised_values
            test = []

        # transform the scale of the data
        scaler, train_scaled, test_scaled = scale(train, test)
        
        scalers[state] = scaler
        
        # save test set
        test_sets[state] = test_scaled
        validation[state] = raw_values
        # fit the model
        lstm_model = fit_lstm(lstm_model, train_scaled, 1, 3000)
    
    return lstm_model, test_sets, validation, scalers


def test_model(lstm_model, testing_n, test_sets, validation, scalers):
    all_predictions = {}
    results = {}

    for state in training_features['location_name'].unique():
        # walk-forward validation on the test data
        predictions = list()
        
        scaler = scalers[state]
        test_scaled = test_sets[state]
        raw_values = validation[state]

        for i in range(len(test_scaled)):
            # make one-step forecast
            X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
            yhat = forecast_lstm(lstm_model, 1, X)
            # invert scaling
            yhat = invert_scale(scaler, X, yhat)
            # invert differencing
            yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
            # store forecast
            predictions.append(yhat)
        # report performance
        rmse = np.sqrt(mean_squared_error(raw_values[-testing_n:], predictions))
        mean_acc = np.mean(raw_values[-testing_n:])
        prop = rmse/mean_acc

        mae = mean_absolute_error(raw_values[-testing_n:], predictions)
        prop2 = mae/mean_acc

        results['state'] = results.get('state', []) + [state]
        results['RMSE'] = results.get('RMSE', []) + [rmse]
        results['RMSE Prop'] = results.get('RMSE Prop', []) + [prop]
        results['MAE'] = results.get('MAE', []) + [mae]
        results['MAE Prop'] = results.get('MAE Prop', []) + [prop2]

        all_predictions[state] = predictions
        
        
    return pd.DataFrame(results).sort_values('MAE Prop'), all_predictions


def get_r(validation, predictions, testing_n):
    y_pred = []
    y_test = []

    for state in validation.keys():
        y_test += list(validation[state][-testing_n:])
        y_pred += list(all_predictions[state])

    return pearsonr(y_test, y_pred)

#### Read-in IHME and Claims data

In [4]:
ihme = pd.read_csv('data/2020_05_08/Hospitalization_all_locs.csv').drop('V1', axis = 1)
claims = pd.read_csv('data/weekly-unemployment-claims.csv')

In [5]:
# Need to decide what locations to deal with
keep_locations = (ihme['location_name'].isin(names.state_names)) | ihme['location_name'].apply(lambda d: d[-2:] in names.state_abbs)

#### Process IHME and Convert to weekly on Saturdays as per claims data

In [6]:
ihme = ihme[ihme['location_name'].isin(names.state_names)][['location_name', 'date'] + names.ihme_cols]
ihme['week ended'] = ihme['date'].apply(lambda d: get_next_weekday(d, 5))

In [7]:
d = {}
for i, col in enumerate(list(ihme)[2:-1]):
    d[col] = 'sum'
        
d['total_tests'] = 'mean'
d['est_infections_mean'] = 'mean'
d['est_infections_lower'] = 'max'
d['est_infections_upper'] = 'min'

In [8]:
ihme_collapsed = ihme.groupby(['location_name', 'week ended'], as_index = False).agg(d)
ihme_collapsed = ihme_collapsed[ihme_collapsed['week ended'].apply(lambda d: datetime.strptime(d, '%Y-%m-%d') > datetime(2020, 1, 1))]

In [9]:
claims = claims.dropna()

In [10]:
claims['week ended'] = claims['Filed week ended'].apply(lambda d: datetime.strptime(d, '%m/%d/%y').strftime('%Y-%m-%d'))
claims = claims[['State', 'week ended'] + names.claim_cols]
claims.columns = ['location_name', 'week ended'] + names.claim_cols

In [11]:
for col in list(claims)[2:-1]:
    print(col)
    claims[col] = claims[col].apply(lambda s: float(s.replace(',', '')))

Initial Claims
Continued Claims
Covered Employment


#### Merge data

In [12]:
df = ihme_collapsed.merge(claims, on=['location_name', 'week ended'], how = 'left')

In [13]:
data = get_ts(df, 2)
data = data.join(pd.get_dummies(data['location_name'])) #.drop('location_name', axis = 1)
data['week num'] = data['week ended'].apply(lambda d: 
                                            (datetime.strptime(d, '%Y-%m-%d') - 
                                             datetime(2020, 2, 22)).days / 7)
data = data[data['week num'] >= 0]

/Users/abdallaha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
week_num_preds = data['week num'][data['week ended'] == datetime(2020, 4, 25).strftime('%Y-%m-%d')].iloc[0] + 0.5

#### Read-in Census Quickfacts data

In [15]:
quickfacts = pd.read_csv('data/Census QuickFacts/combined_use.csv').replace('Z', 0)

In [16]:
for col in list(quickfacts)[1:]:
#     print(col)
    quickfacts[col] = quickfacts[col].apply(lambda s: string_fixer(s)).astype(float)
quickfacts['location_name'] = quickfacts['location_name'].apply(lambda s: s.strip())
quickfacts.columns = [col.strip() for col in list(quickfacts)]

#### Merge with ihme and claims data, then split into present and future data

In [17]:
data = data.merge(quickfacts, how = 'left', on = 'location_name')

In [18]:
pres_data = data[data['week ended'].apply(lambda s: datetime.strptime(s, '%Y-%m-%d') <= datetime(2020, 4, 25))].drop('week ended', axis = 1)
futu_data = data[data['week ended'].apply(lambda s: datetime.strptime(s, '%Y-%m-%d') > datetime(2020, 4, 25))].drop('week ended', axis = 1)

In [19]:
training_features = pres_data.drop(names.claim_cols, axis = 1)
initial_claims = pres_data['Initial Claims']

#### Running the first time to test for 3 week predictions

In [20]:
lstm_model, test_sets, validation, scalers = train_model(training_features, initial_claims, 3)

In [21]:
results, all_predictions = test_model(lstm_model, 3, test_sets, validation, scalers)

In [22]:
get_r(validation, all_predictions, 3)

(0.8585788792202089, 8.683987141101414e-45)

#### Training the model for prediction

In [23]:
lstm_model, test_sets, validation, scalers = train_model(training_features, initial_claims, 1)

### Start of prediction

In [24]:
future = futu_data.drop(names.claim_cols, axis = 1)

weeks = sorted(future['week num'].unique())

initial_diff = {}
actual_state = {}
for state in names.state_names:
    tminus1 = validation[state][-2]
    t = validation[state][-1]
    initial_diff[state] = t - tminus1
    actual_state[state] = t

In [25]:
diff_predictions = {}
for state in names.state_names:
    state_df = future[future['location_name'] == state].drop('location_name', axis = 1)
    scaler = scalers[state]
    
    diff = initial_diff[state]
    
    preds = []
    
    for week in weeks:        
        df = state_df[state_df['week num'] == week]
        df['claim diff last week'] = [diff]
        df['empty col'] = [0]
        
        assert len(df) == 1
        
        values = df.values
        X_scaled = scaler.transform(values)
        X = X_scaled[0, :-1]
        
        yhat = forecast_lstm(lstm_model, 1, X)
        yhat = invert_scale(scaler, X, yhat)
        
        diff = yhat
        preds.append(yhat)
    
    diff_predictions[state] = preds

/Users/abdallaha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/abdallaha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/abdallaha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [26]:
actual_predictions = {}
for state, diffs in diff_predictions.items():
    curr = validation[state][-1]
    final = []
    for diff in diffs:
        new_curr = curr + diff
        
        curr = max(0, new_curr) 
        
        final.append(curr)
    actual_predictions[state] = final